Отчет по ДЗ:
Добился accuracy выше 0.95. Лучшие результаты показали стеммизированные токены (пробовал стеммы, лемы и просто отфильтрованные токены). Сильно повлияло на качество количество слов, которые мы добавляем в словарь. Связано это с тем, что тогда вектор, котоыре мы передаем, содержит меньшее кол-во нулей (уменьшаем паддинг). Правда не понял, почему сильно ухудшило качество сети, когда я вообще перешел к малым значениям вектора (20 вместо 50). По идее, это привело к уменьшению паддинга, но на практике оч плохо отработало.

Использование предобученного эмбединга w2vпоказало худшие результаты, чем обучение слоя в керас.



Вопросы по дз.
1. У меня не получилось использовать несколько слоев в сети (один с биграммами, другой с триграммами, к примеру) Есть ли пример кода, как это можно реализовать? Хотелось бы проверить, как это скажется на результатах.
2. У меня возникли проблемы с использованием предобученного эмбединга. Я просто методом тыка пришел к тому, что это заработало. И у меня много вопросов, как это вообще работате =)
Я учу w2v на токенах слов. Получаю вектора, в зависимости от семантической близости, тут все ясно. Потом я использую эти веса в эмбединг слое, но передаю туда не слова или твиты, а мешки слов. Откуда он знает, что это такое, как он веса применяет? Простыми словами он обучился, что слово трамп имеет определленный вектор размера d, а я ему на вход в сети подаю вектор [0,0,0,0,0,2,44]. Как он вообще применяет сови веса к такому вектору?


In [ ]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd

#df_train = pd.read_csv("data/train.csv")
#df_test = pd.read_csv("data/test.csv")
#df_val = pd.read_csv("data/val.csv")
df=pd.read_csv('tweets.csv', index_col=0)

In [ ]:
df['label'].isnull().value_counts()

False    31962
True     17197
Name: label, dtype: int64

In [ ]:
df=df[:31962]

In [ ]:
df

,label,tweet,clear_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
id,,,,,,,
1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"['when', 'father', 'is', 'dysfunctional', 'and...","['father', 'dysfunctional', 'selfish', 'drags'...","['father', 'dysfunct', 'selfish', 'drag', 'kid...","['father', 'dysfunctional', 'selfish', 'drag',..."
2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use because they...,"['thanks', 'for', 'lyft', 'credit', 'can', 'no...","['thanks', 'lyft', 'credit', 'use', 'offer', '...","['thank', 'lyft', 'credit', 'use', 'offer', 'w...","['thank', 'lyft', 'credit', 'use', 'offer', 'w..."
3,0.0,bihday your majesty,bihday your majesty,"['bihday', 'your', 'majesty']","['bihday', 'majesty']","['bihday', 'majesti']","['bihday', 'majesty']"
4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in your,"['model', 'love', 'you', 'take', 'with', 'you'...","['model', 'love', 'take', 'time']","['model', 'love', 'take', 'time']","['model', 'love', 'take', 'time']"
5,0.0,factsguide: society now #motivation,factsguide society now motivation,"['factsguide', 'society', 'now', 'motivation']","['factsguide', 'society', 'motivation']","['factsguid', 'societi', 'motiv']","['factsguide', 'society', 'motivation']"
...,...,...,...,...,...,...,...
31958,0.0,ate @user isz that youuu?ðððððð...,ate isz that youuu,"['ate', 'isz', 'that', 'youuu']","['ate', 'isz', 'youuu']","['ate', 'isz', 'youuu']","['eat', 'isz', 'youuu']"
31959,0.0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...,"['to', 'see', 'nina', 'turner', 'on', 'the', '...","['see', 'nina', 'turner', 'airwaves', 'trying'...","['see', 'nina', 'turner', 'airwav', 'tri', 'wr...","['see', 'nina', 'turner', 'airwaves', 'try', '..."
31960,0.0,listening to sad songs on a monday morning otw...,listening to sad songs on monday morning otw t...,"['listening', 'to', 'sad', 'songs', 'on', 'mon...","['listening', 'sad', 'songs', 'monday', 'morni...","['listen', 'sad', 'song', 'monday', 'morn', 'o...","['listen', 'sad', 'songs', 'monday', 'morning'..."


### Предобработка

In [ ]:

corpus=''

In [ ]:
for i,values in df["tweet_stemmed"].iteritems():
  res = [element.strip("'[]") for element in str(values).split(", ")]
  train_corpus = " ".join(res)
  corpus+=train_corpus
  #train_corpus.append(res)
  #print(res)

In [ ]:
tweets_list=[]
for i,values in df["tweet_token_filtered"].iteritems():
  res = [element.strip("'[]") for element in str(values).split(", ")]
  tweets_list.append(res) 
  

In [ ]:
stem_text=[]
for i,values in df["tweet_stemmed"].iteritems():
  res = [element.strip("'[]") for element in str(values).split(", ")]
  train_corpus = " ".join(res)
  stem_text.append(train_corpus)

In [ ]:
lem_text=[]
for i,values in df["tweet_lemmatized"].iteritems():
  res = [element.strip("'[]") for element in str(values).split(", ")]
  train_corpus = " ".join(res)
  lem_text.append(train_corpus)

In [ ]:
df['stem_text']=stem_text

In [ ]:
df['lem_text']=lem_text

In [ ]:
df

,label,tweet,clear_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized,stem_text,lem_text
id,,,,,,,,,
1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"['when', 'father', 'is', 'dysfunctional', 'and...","['father', 'dysfunctional', 'selfish', 'drags'...","['father', 'dysfunct', 'selfish', 'drag', 'kid...","['father', 'dysfunctional', 'selfish', 'drag',...",father dysfunct selfish drag kid dysfunct run,father dysfunctional selfish drag kid dysfunct...
2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use because they...,"['thanks', 'for', 'lyft', 'credit', 'can', 'no...","['thanks', 'lyft', 'credit', 'use', 'offer', '...","['thank', 'lyft', 'credit', 'use', 'offer', 'w...","['thank', 'lyft', 'credit', 'use', 'offer', 'w...",thank lyft credit use offer wheelchair van pdx...,thank lyft credit use offer wheelchair vans pd...
3,0.0,bihday your majesty,bihday your majesty,"['bihday', 'your', 'majesty']","['bihday', 'majesty']","['bihday', 'majesti']","['bihday', 'majesty']",bihday majesti,bihday majesty
4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in your,"['model', 'love', 'you', 'take', 'with', 'you'...","['model', 'love', 'take', 'time']","['model', 'love', 'take', 'time']","['model', 'love', 'take', 'time']",model love take time,model love take time
5,0.0,factsguide: society now #motivation,factsguide society now motivation,"['factsguide', 'society', 'now', 'motivation']","['factsguide', 'society', 'motivation']","['factsguid', 'societi', 'motiv']","['factsguide', 'society', 'motivation']",factsguid societi motiv,factsguide society motivation
...,...,...,...,...,...,...,...,...,...
31958,0.0,ate @user isz that youuu?ðððððð...,ate isz that youuu,"['ate', 'isz', 'that', 'youuu']","['ate', 'isz', 'youuu']","['ate', 'isz', 'youuu']","['eat', 'isz', 'youuu']",ate isz youuu,eat isz youuu
31959,0.0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...,"['to', 'see', 'nina', 'turner', 'on', 'the', '...","['see', 'nina', 'turner', 'airwaves', 'trying'...","['see', 'nina', 'turner', 'airwav', 'tri', 'wr...","['see', 'nina', 'turner', 'airwaves', 'try', '...",see nina turner airwav tri wrap mantl genuin h...,see nina turner airwaves try wrap mantle genui...
31960,0.0,listening to sad songs on a monday morning otw...,listening to sad songs on monday morning otw t...,"['listening', 'to', 'sad', 'songs', 'on', 'mon...","['listening', 'sad', 'songs', 'monday', 'morni...","['listen', 'sad', 'song', 'monday', 'morn', 'o...","['listen', 'sad', 'songs', 'monday', 'morning'...",listen sad song monday morn otw work sad,listen sad songs monday morning otw work sad


In [ ]:
df['tweets_list']=tweets_list

In [ ]:
df

,label,tweet,clear_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized,stem_text,lem_text,tweets_list
id,,,,,,,,,,
1,0.0,@user when a father is dysfunctional and is s...,when father is dysfunctional and is so selfish...,"['when', 'father', 'is', 'dysfunctional', 'and...","['father', 'dysfunctional', 'selfish', 'drags'...","['father', 'dysfunct', 'selfish', 'drag', 'kid...","['father', 'dysfunctional', 'selfish', 'drag',...",father dysfunct selfish drag kid dysfunct run,father dysfunctional selfish drag kid dysfunct...,"[father, dysfunctional, selfish, drags, kids, ..."
2,0.0,@user @user thanks for #lyft credit i can't us...,thanks for lyft credit cannot use because they...,"['thanks', 'for', 'lyft', 'credit', 'can', 'no...","['thanks', 'lyft', 'credit', 'use', 'offer', '...","['thank', 'lyft', 'credit', 'use', 'offer', 'w...","['thank', 'lyft', 'credit', 'use', 'offer', 'w...",thank lyft credit use offer wheelchair van pdx...,thank lyft credit use offer wheelchair vans pd...,"[thanks, lyft, credit, use, offer, wheelchair,..."
3,0.0,bihday your majesty,bihday your majesty,"['bihday', 'your', 'majesty']","['bihday', 'majesty']","['bihday', 'majesti']","['bihday', 'majesty']",bihday majesti,bihday majesty,"[bihday, majesty]"
4,0.0,#model i love u take with u all the time in ...,model love you take with you all the time in your,"['model', 'love', 'you', 'take', 'with', 'you'...","['model', 'love', 'take', 'time']","['model', 'love', 'take', 'time']","['model', 'love', 'take', 'time']",model love take time,model love take time,"[model, love, take, time]"
5,0.0,factsguide: society now #motivation,factsguide society now motivation,"['factsguide', 'society', 'now', 'motivation']","['factsguide', 'society', 'motivation']","['factsguid', 'societi', 'motiv']","['factsguide', 'society', 'motivation']",factsguid societi motiv,factsguide society motivation,"[factsguide, society, motivation]"
...,...,...,...,...,...,...,...,...,...,...
31958,0.0,ate @user isz that youuu?ðððððð...,ate isz that youuu,"['ate', 'isz', 'that', 'youuu']","['ate', 'isz', 'youuu']","['ate', 'isz', 'youuu']","['eat', 'isz', 'youuu']",ate isz youuu,eat isz youuu,"[ate, isz, youuu]"
31959,0.0,to see nina turner on the airwaves trying to...,to see nina turner on the airwaves trying to w...,"['to', 'see', 'nina', 'turner', 'on', 'the', '...","['see', 'nina', 'turner', 'airwaves', 'trying'...","['see', 'nina', 'turner', 'airwav', 'tri', 'wr...","['see', 'nina', 'turner', 'airwaves', 'try', '...",see nina turner airwav tri wrap mantl genuin h...,see nina turner airwaves try wrap mantle genui...,"[see, nina, turner, airwaves, trying, wrap, ma..."
31960,0.0,listening to sad songs on a monday morning otw...,listening to sad songs on monday morning otw t...,"['listening', 'to', 'sad', 'songs', 'on', 'mon...","['listening', 'sad', 'songs', 'monday', 'morni...","['listen', 'sad', 'song', 'monday', 'morn', 'o...","['listen', 'sad', 'songs', 'monday', 'morning'...",listen sad song monday morn otw work sad,listen sad songs monday morning otw work sad,"[listening, sad, songs, monday, morning, otw, ..."


In [ ]:
corpus

'father dysfunct selfish drag kid dysfunct runthank lyft credit use offer wheelchair van pdx disapoint getthankbihday majestimodel love take timefactsguid societi motivhuge fan fare big talk leav chao pay disput get allshowandnogocamp tomorrow danninext school year year exam think school exam hate imagin actorslif revolutionschool girllove land allin cav champion cleveland clevelandcavaliwelcom grireland consum price index mom climb previou may blog silver gold forexselfish orlando standwithorlando pulseshoot orlandoshoot biggerproblem selfish heabreak valu loveget see daddi today day gettingfcnn call michigan middl school build wall chant tcotcomment australia opkillingbay seashepherd helpcovedolphin thecov helpcovedolphinouch junior angri got junior yugyoem omgthank paner thank positretweet agrefriday smile around via ig user cooki make peoplknow essenti oil made chemiceuro peopl blame ha conced goal fat rooney gave away free kick know bale hitsad littl dude badday coneofsham cat pis

In [ ]:
max_words = 3500
max_len = 50
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
len(tokens)

211581

Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [ ]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [ ]:
len(tokens_filtered)

211581

In [ ]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [ ]:
tokens_filtered_top[:10]

['love', 'day', 'happi', 'like', 'get', 'go', 'life', 'thank', 'make', 'peopl']

In [ ]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [ ]:
import numpy as np
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(str(text).lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [ ]:
train=np.asarray([text_to_sequence(text, max_len) for text in df["stem_text"]], dtype=np.int32)

In [ ]:
train.shape

(31962, 50)

In [ ]:
train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,   18, 2006, 1721,  113,  154], dtype=int32)

# Keras model



Как делать несколько слоев с фильтром? допустим взять 2, 3 и 4 величину фильтра. Как именно технически это прописать, есть пример кода? Не смог добавить в код с методичке еще доп фильтров


In [ ]:
num_classes = 2
y = keras.utils.to_categorical(df["label"], num_classes)
#y_val = keras.utils.to_categorical(df_val["class"], num_classes)

In [ ]:
x_train,x_val,y_train,y_val=train_test_split(train,y)

In [ ]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len
                   ))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
43/43 [==============================] - 8s 178ms/step - loss: 0.3306 - accuracy: 0.9293 - val_loss: 0.2406 - val_accuracy: 0.9258
Epoch 2/20
43/43 [==============================] - 7s 173ms/step - loss: 0.1777 - accuracy: 0.9397 - val_loss: 0.1372 - val_accuracy: 0.9529
Epoch 3/20
43/43 [==============================] - 7s 172ms/step - loss: 0.0980 - accuracy: 0.9650 - val_loss: 0.1341 - val_accuracy: 0.9587
Epoch 4/20
43/43 [==============================] - 7s 172ms/step - loss: 0.0685 - accuracy: 0.9758 - val_loss: 0.1484 - val_accuracy: 0.9583


In [ ]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

16/16 [==============================] - 1s 40ms/step - loss: 0.1317 - accuracy: 0.9582


Test score: 0.13173900544643402
Test accuracy: 0.9582029581069946


In [ ]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

45/45 [==============================] - 0s 8ms/step


In [ ]:
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec

In [ ]:
model = Word2Vec(df['tweets_list'], size=200, window=5, min_count=1, sg = 1, hs = 0, negative = 10, workers= 32, seed = 34)
model.train(df['tweets_list'], total_examples=len(df['tweets_list']), epochs=20)

(4717620, 4870840)

In [ ]:
result = model.most_similar(positive='trump')
result

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('donald', 0.6298798322677612),
 ('suppoer', 0.6043557524681091),
 ('dumptrump', 0.582619309425354),
 ('remarks', 0.5703113675117493),
 ('fuhered', 0.5627217292785645),
 ('commie', 0.5579087138175964),
 ('republican', 0.5570115447044373),
 ('cuck', 0.5529880523681641),
 ('ally', 0.5496256351470947),
 ('yuge', 0.5492163896560669)]

In [ ]:
matrix=model.wv.get_keras_embedding(train_embeddings=False)

In [ ]:
matrix

In [ ]:
model = Sequential()
model.add(matrix)
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
43/43 [==============================] - 8s 179ms/step - loss: 0.3946 - accuracy: 0.8404 - val_loss: 0.2644 - val_accuracy: 0.9258
Epoch 2/20
43/43 [==============================] - 7s 171ms/step - loss: 0.2465 - accuracy: 0.9300 - val_loss: 0.2535 - val_accuracy: 0.9258
Epoch 3/20
43/43 [==============================] - 7s 172ms/step - loss: 0.2388 - accuracy: 0.9300 - val_loss: 0.2442 - val_accuracy: 0.9258
Epoch 4/20
43/43 [==============================] - 7s 173ms/step - loss: 0.2292 - accuracy: 0.9300 - val_loss: 0.2324 - val_accuracy: 0.9258
Epoch 5/20
43/43 [==============================] - 7s 173ms/step - loss: 0.2174 - accuracy: 0.9300 - val_loss: 0.2199 - val_accuracy: 0.9258
Epoch 6/20
43/43 [==============================] - 7s 172ms/step - loss: 0.2040 - accuracy: 0.9300 - val_loss: 0.2085 - val_accuracy: 0.9258
Epoch 7/20
43/43 [==============================] - 7s 173ms/step - loss: 0.1917 - accuracy: 0.9300 - val_loss: 0.2007 - val_accuracy: 0.9258
Epoch 

In [ ]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

16/16 [==============================] - 1s 65ms/step - loss: 0.1768 - accuracy: 0.9438


Test score: 0.17677715420722961
Test accuracy: 0.9438117742538452
